In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 26
data_dim = 3
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 883

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0, 1)) # 데이터 일반화

# xy = np.loadtxt('./v3data/train_v3_data_cct.csv', delimiter=',')
# cct, cas_swr, 446to477, uvb (883행 4열)
xy = np.loadtxt('./v7data/train_v7_data.csv', delimiter=',')

x = scaler.fit_transform(xy[:, [0, 2, 3]]) # x = 맨 마지막 ptmt 제외 모든 것
y = scaler2.fit_transform(xy[:, [1]]) 

# build a dataset
dataX = []
dataY = []

# print(len(x))
# print(len(y))
# print(x[0])
# print(y[0])

for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)
#     print(np.shape(_x))

print(np.shape(dataX))
print(np.shape(dataY))

data = '2018-05-04'
# xy2 = np.loadtxt('./v3data/'+ data +'.csv',delimiter=',')
# xy2 = np.loadtxt('./v3data/test_v3_data_cct_180331.csv', delimiter=',')
# xy2 = np.loadtxt('./v3data/new/20180421.csv', delimiter=',') # 예측할 날짜
xy2 = np.loadtxt('./v7data/test_v7_data.csv', delimiter=',')

# x2 = scaler.fit_transform(xy2) # 데이터 없을때 (데이터란 ptmt 값)
x2 = scaler.fit_transform(xy2[:, [0, 2, 3]]) # 데이터 있을때
y2 = scaler2.fit_transform(xy2[:, [1]]) #데이터 있을때

# print(np.shape(x2))

dataX2 = []
dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
    _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
    dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

# print(train_size)
# print(test_size)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

# print(np.shape(trainX))

(22932, 26, 3)
(22932, 1)


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
    dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 5189.48095703125
[step: 1] loss: 3140.64892578125
[step: 2] loss: 1693.6829833984375
[step: 3] loss: 687.488037109375
[step: 4] loss: 330.1980895996094
[step: 5] loss: 617.039306640625
[step: 6] loss: 760.6430053710938
[step: 7] loss: 608.018798828125
[step: 8] loss: 371.5423583984375
[step: 9] loss: 192.1704559326172
[step: 10] loss: 103.29158782958984
[step: 11] loss: 84.25955963134766
[step: 12] loss: 102.51351928710938
[step: 13] loss: 131.05506896972656
[step: 14] loss: 152.63963317871094
[step: 15] loss: 159.20982360839844
[step: 16] loss: 149.83486938476562
[step: 17] loss: 128.17083740234375
[step: 18] loss: 99.98747253417969
[step: 19] loss: 71.18486785888672
[step: 20] loss: 46.4925651550293
[step: 21] loss: 28.828649520874023
[step: 22] loss: 19.174776077270508
[step: 23] loss: 16.810956954956055
[step: 24] loss: 19.781831741333008
[step: 25] loss: 25.49486541748047
[step: 26] loss: 31.348669052124023
[step: 27] loss: 35.277244567871094
[step: 28] loss: 36.10

[step: 227] loss: 0.6482344269752502
[step: 228] loss: 0.6476441025733948
[step: 229] loss: 0.6470707654953003
[step: 230] loss: 0.6465139985084534
[step: 231] loss: 0.6459732055664062
[step: 232] loss: 0.645447850227356
[step: 233] loss: 0.6449390649795532
[step: 234] loss: 0.6444440484046936
[step: 235] loss: 0.6439629793167114
[step: 236] loss: 0.6434950232505798
[step: 237] loss: 0.6430423259735107
[step: 238] loss: 0.6426010131835938
[step: 239] loss: 0.6421732306480408
[step: 240] loss: 0.6417570114135742
[step: 241] loss: 0.6413536071777344
[step: 242] loss: 0.6409614682197571
[step: 243] loss: 0.6405815482139587
[step: 244] loss: 0.640209972858429
[step: 245] loss: 0.6398496627807617
[step: 246] loss: 0.6395002007484436
[step: 247] loss: 0.6391599774360657
[step: 248] loss: 0.6388290524482727
[step: 249] loss: 0.6385077834129333
[step: 250] loss: 0.6381944417953491
[step: 251] loss: 0.6378909349441528
[step: 252] loss: 0.6375947594642639
[step: 253] loss: 0.637307345867157
[ste

[step: 450] loss: 0.6186548471450806
[step: 451] loss: 0.6186008453369141
[step: 452] loss: 0.6185476183891296
[step: 453] loss: 0.6184946298599243
[step: 454] loss: 0.6184418797492981
[step: 455] loss: 0.6183887720108032
[step: 456] loss: 0.6183362007141113
[step: 457] loss: 0.6182838082313538
[step: 458] loss: 0.618231475353241
[step: 459] loss: 0.6181796193122864
[step: 460] loss: 0.6181278824806213
[step: 461] loss: 0.6180760264396667
[step: 462] loss: 0.6180243492126465
[step: 463] loss: 0.6179726123809814
[step: 464] loss: 0.6179211735725403
[step: 465] loss: 0.6178702116012573
[step: 466] loss: 0.6178191900253296
[step: 467] loss: 0.6177684664726257
[step: 468] loss: 0.617717981338501
[step: 469] loss: 0.617667555809021
[step: 470] loss: 0.6176171898841858
[step: 471] loss: 0.6175669431686401
[step: 472] loss: 0.6175169944763184
[step: 473] loss: 0.6174671649932861
[step: 474] loss: 0.6174176335334778
[step: 475] loss: 0.6173679232597351
[step: 476] loss: 0.6173183917999268
[ste

[step: 673] loss: 0.6102886199951172
[step: 674] loss: 0.6102643013000488
[step: 675] loss: 0.6102398037910461
[step: 676] loss: 0.6102156639099121
[step: 677] loss: 0.6101917028427124
[step: 678] loss: 0.6101678609848022
[step: 679] loss: 0.6101438403129578
[step: 680] loss: 0.6101201772689819
[step: 681] loss: 0.6100963354110718
[step: 682] loss: 0.6100727319717407
[step: 683] loss: 0.6100491285324097
[step: 684] loss: 0.6100255846977234
[step: 685] loss: 0.6100019812583923
[step: 686] loss: 0.6099786758422852
[step: 687] loss: 0.609955370426178
[step: 688] loss: 0.6099321246147156
[step: 689] loss: 0.6099090576171875
[step: 690] loss: 0.6098862290382385
[step: 691] loss: 0.6098632216453552
[step: 692] loss: 0.609840452671051
[step: 693] loss: 0.6098176836967468
[step: 694] loss: 0.6097949147224426
[step: 695] loss: 0.6097725033760071
[step: 696] loss: 0.6097500324249268
[step: 697] loss: 0.6097278594970703
[step: 698] loss: 0.6097054481506348
[step: 699] loss: 0.609683096408844
[ste

In [5]:
# Plot predictions
plt.title(data + ' predict')
plt.plot(dataY2, label='raw data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
# plt.ylabel("photometric")
# plt.ylabel("cct")
plt.ylabel("CAS_SWR")
# plt.ylabel("446to477")
# plt.ylabel("UV-B")
plt.xticks(np.arange(0, 883, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
# plt.yticks(np.arange(0, 140000, step=10000)) # ptmt
# plt.yticks(np.arange(0, 60000, step=10000)) # cct
# plt.yticks(np.arange(0, 2.0, step=0.2)) # uvb
# plt.ylim(0, 100) #swr, 446to477
plt.grid(True)
plt.legend(loc='upper right')
plt.show()